# Initiate a in place restore of a MSSQL Database
In Rubrik, a restore is a in place operation. The original database on the original SQL Server instance is overwritten with the point in time provided. This is very important to understand as you could cause a production outage recovering a database when you mean to make a copy of a database. For that use export. Restore should only be used as a disaster recovery process.  

New-RscMssqlRestore has multiple parameters, but the important ones to understand for most workflows are:
- RscMssqlDatabase: This will be the object returned from Get-RscMssqlDatabase
- Recovery Time: For the point in time value, there are a few different possibilities. 
    - **latest**: This will tell Rubrik to export the database to the latest recovery point Rubrik knows about. This will include the last full and any logs to get to the latest recovery point
    - **lastfull**: This will tell Rubrik to restore back to the last full backup it has
    - **RestoreTime**: This can be done in 1 of 2 formats
        - Format: (HH:MM:SS.mmm) or (HH:MM:SS.mmmZ) - Restores the database back to the local or UTC time (respectively) at the point in time specified within the last 24 hours
        - Format: Any valid <value> that PS Get-Date supports in: "Get-Date -Date <Value>"
            - Example: "2018-08-01T02:00:00.000Z" restores back to 2AM on August 1, 2018 UTC.
    This cmdlet will run Get-RscMssqlDatabaseRecoveryPoint
- FinishRecovery: Ommitting this switch will restore the database into NORECOVERY mode. 

# Workflow
To get information about the SQL Server Instance, you need to do the following
1. Use **Connect-RSC** to Connect to RSC
1. Use **Get-RscCluster** to get the cluster id that the SQL Server is registered. This is akin to the datacenter that the SQL Server is located in. While you can run the next command without the cluster ID, you may get more records back than expected. This is because your SQL Server Host could exist on 2 different cluster. Once as a replicated source and once as the replicated target. 
1. Use **Get-RscMssqlInstance** to get the instance id of the SQL Server based on the name of the instance and the cluster where it is located. 
1. Use **Get-RscMssqlDatabase** to get the database ID based on the database name and the SQL Server Instance. 
1. Use **Get-RscMssqlDatabaseRecoveryPoint** to get the fully formatted date that corresponds to the recovery point selected. 
1. Use **New-RscMssqlRestore** to start an in place restore of the database to the point in time provided. 
1. Use **Get-RscRequest** to track the request and know when it completes. ***DOES NOT EXIST YET***
1. Use **Disconnect-RSC** to disconnect from RSC
 new-rscmssqllogBackup -RscMssqlDatabase $mssqldatabase  

## Connect to RSC

In [1]:
Set-Location '/Users/Chris.Lumnah/Library/CloudStorage/GoogleDrive-chris.lumnah@rubrik.com/My Drive/1.Projects/Powershell SDK/rubrik-powershell-sdk'
Import-Module ./output/rubriksecuritycloud -force
# Import-Module RubrikSecurityCloud
Connect-RSC


ClientId                                AuthenticationState SdkSchemaVersion ServerVersion
--------                                ------------------- ---------------- -------------
client|kB5og68XXrFlfXcvWhn3jlHdf1OKyJCB          AUTHORIZED v20231108-58     v20231108-58



## Get the Cluster ID

In [2]:
$ClusterName = "vault-r-london"
$RscCluster = Get-RscCluster -Name $ClusterName
$RscCluster


LicensedProducts                : 
PauseStatus                     : NOT_PAUSED
ProductType                     : 
RegisteredMode                  : 
Status                          : CONNECTED
SubStatus                       : DEFAULT
SystemStatus                    : OK
Type                            : ON_PREM
ConnectivityLastUpdated         : 
DefaultAddress                  : 
DefaultPort                     : 
EncryptionEnabled               : 
EstimatedRunway                 : 
Id                              : 39b92c18-d897-4b55-a7f9-17ff178616d0
IsHealthy                       : 
LastConnectionTime              : 
Name                            : vault-r-london
NoSqlWorkloadCount              : 
PassesConnectivityCheck         : 
RegistrationTime                : 
SnapshotCount                   : 
SystemStatusMessage             : 1 Node Down.
Timezone                        : 
Version                         : 9.0.0-24610
ActivitySeriesConnection        : 
AllOrgs         

## Get the Instance Information

In [3]:
$HostName = "rp-sql19s-001.perf.rubrik.com"
$RscHost = Get-RscHost -Name $HostName -OsType Windows

$RscMssqlInstance = Get-RscMssqlInstance -RscHost $RscHost -RscCluster $RscCluster
$RscMssqlInstance


AuthorizedOperations        : 
ObjectType                  : PHYSICAL_HOST
OsType                      : 
SlaAssignment               : UNASSIGNED
ConfiguredSlaDomain         : 
EffectiveRetentionSlaDomain : 
EffectiveSlaDomain          : RubrikSecurityCloud.Types.GlobalSlaReply
PendingSla                  : 
ReplicatedObjects           : 
AgentId                     : {e0e2af29-6566-460f-986b-4ba31619316f}
CbtStatus                   : Enabled
CdmId                       : 93c5aed0-2140-4c3a-a464-d57b8a8dcfbb
CdmLink                     : 
DefaultCbt                  : 
Id                          : 9a2bc8af-3e59-5036-bfc4-a6252fd968e9
IpAddresses                 : 
IsArchived                  : False
IsChangelistEnabled         : False
IsOracleHost                : False
Name                        : rp-sql19s-001.perf.rubrik.com
NasMigrationInfo            : 
NasVendorType               : 
NumWorkloadDescendants      : 9
OsName                      : 
ReplicatedObjectCount       : 

## Get the Database Information

In [4]:
$DatabaseName = "AdventureWorks2019"
$RscMssqlDatabase = Get-RscMssqlDatabase -Name $DatabaseName -RscMssqlInstance $RscMssqlInstance

## Get the fully formatted recovery point

In [5]:
$RecoveryDateTime = Get-RscMssqlDatabaseRecoveryPoint -RscMssqlDatabase $RscMssqlDatabase -Latest
$RecoveryDateTime

2023-11-29T00:40:32.000Z


## Intiate an in place restore of a database using the last snapshot

In [6]:
New-RscMssqlRestore -RscMssqlDatabase $RscMssqlDatabase -RecoveryDateTime $RecoveryDateTime -FinishRecovery


EndTime   : 
Id        : RESTORE_MSSQL_DB_138e0fbc-c0f8-4df2-97b4-3338258e49c1_9ab99e17-fb5d-426c-9d9b-deddabc3c
            fe4:::0
NodeId    : 
Progress  : 
StartTime : 
Status    : QUEUED
Error     : 
Links     : 




## Track the status of the restore

---

# Disconnect from RSC
It is important to have each script you create end by disconnecting from RSC. If you do not, then you will run into issues with token authentication exhaustion. RSC allows a Service Account to have 10 concurrent open connections to RSC. Additional requests will be denied until existing connections are closed. 

You can easily disconnect your connection by running the below statement. 

In [ ]:
Disconnect-Rsc